In [1]:
import plotly.graph_objects as go
import pickle
import numpy as np
import os
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
labels = ['relb','fish','bash','time']
colors = ['#0ad2ff', '#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#0fffdb']

In [3]:

data_folder = 'data'
case_folder = 'with_wall_rpi'
relb_files = [f'{i}relb.pkl' for i in range(7)]
fish_files = [f'{i}fish.pkl' for i in range(7)]
time_files = [f'{i}time.pkl' for i in range(7)]
bash_files = [f'{i}bash.pkl' for i in range(7)]



In [4]:
def load(filename):
    # open the file and load the stored data using pickle
    with open(filename, 'rb') as f:
        return pickle.load(f)
def unpack_data(files, wall=False):
    if wall==True:
        obs_list = []
        rep_list = []
        wall_list =[]
        for file in files:
            path = os.path.join(os.getcwd(), data_folder, case_folder, file)
            wall, rep, obs = load(path)
            wall_list.append(wall)
            rep_list.append(rep)
            obs_list.append(obs)
        return wall_list, rep_list, obs_list
    else:
        obs_list = []
        rep_list = []
        for file in files:
            path = os.path.join(os.getcwd(), data_folder, case_folder, file)
            rep, obs = load(path)
            rep_list.append(rep)
            obs_list.append(obs)
        return rep_list, obs_list

In [5]:
fish_wall_list, fish_rep_list, fish_obs_list = unpack_data(fish_files, wall=True)
time_wall_list, time_rep_list, time_obs_list = unpack_data(time_files, wall=True)
bash_wall_list, bash_rep_list, bash_obs_list = unpack_data(bash_files, wall=True)
relb_wall_list, relb_rep_list, relb_obs_list = unpack_data(relb_files, wall=True)
observed_times = [relb_obs_list, fish_obs_list, bash_obs_list, time_obs_list]
reported_times = [relb_rep_list, fish_rep_list, bash_rep_list, time_rep_list]
wall_times = [relb_wall_list, fish_wall_list, bash_wall_list, time_wall_list]

In [29]:
from scipy.stats import ks_2samp
for i in range(6):
    print(ks_2samp(fish_obs_list[i], relb_obs_list[i], alternative='two-sided'))
    print(ks_2samp(fish_obs_list[i]-np.mean(fish_obs_list[i]), relb_obs_list[i]-np.mean(relb_obs_list[i]), alternative='two-sided'))


KstestResult(statistic=0.093, pvalue=0.00034775170271509687)
KstestResult(statistic=0.359, pvalue=1.2295379067920271e-57)
KstestResult(statistic=0.352, pvalue=2.231788601339735e-55)
KstestResult(statistic=0.563, pvalue=2.165237789572251e-146)
KstestResult(statistic=0.457, pvalue=1.5667756082255056e-94)
KstestResult(statistic=0.597, pvalue=6.036709295933908e-166)
KstestResult(statistic=0.587, pvalue=4.8952441829758626e-160)
KstestResult(statistic=0.437, pvalue=3.510978890942674e-86)
KstestResult(statistic=0.231, pvalue=8.470621055424192e-24)
KstestResult(statistic=0.095, pvalue=0.00023854664270315346)
KstestResult(statistic=0.56, pvalue=9.753426185494649e-145)
KstestResult(statistic=0.504, pvalue=6.683182560861083e-116)


In [6]:
def quadro_plot(fig, data, labels, time_type, start = 0, end = 4000):
    for i, (data_list, group_name) in enumerate(zip(data, labels)):
        for j, (values, color) in enumerate(zip(data_list, colors)):
            fig.add_trace(go.Histogram(x=values, legendgroup=i, marker_color = color, legendgrouptitle_text = group_name + " (num - mean)", name=f'{j} proc - {np.mean(values):.2f}', xbins=dict(size = 10, start = start, end = end)), row=i+1, col=1)

        fig.update_layout(
            #title=f'Run-times distribution ({time_type})',
            xaxis_title='millis',
            yaxis_title='count',
            barmode='overlay',
            bargap=0.1
        )

        fig.update_traces(opacity=0.75)

In [214]:
fig = make_subplots(rows=4, subplot_titles=labels, shared_xaxes=True)
fig.update_layout(font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    ))
quadro_plot(fig, observed_times, labels, "obs")
fig.update_annotations(font_size=12)
fig.write_image("plot7.pdf", scale = 1.0, width = 550, height = 680)
fig.show()

In [227]:
fig1 = make_subplots(rows=4, subplot_titles=labels, shared_xaxes=True)
fig1.update_layout(font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    ))
quadro_plot(fig1, wall_times, labels, "wall", start = 3300, end= 7500)
fig1.update_annotations(font_size=12)
fig1.write_image("plot8.pdf", scale = 1.0, width = 550, height = 680)
fig1.show()

In [179]:
def main_plot(fig, data_list, API_name, time_type, par_proc=-1, col=None, row=None, start = 2000, end = 4000):
    if par_proc == -1:
        for j, (values, color) in enumerate(zip(data_list, colors)):
            fig.add_trace(go.Histogram(x=values, legendgroup=row, marker_color=color, legendgrouptitle_text = time_type + " (num - mean)",  name=f'{j} proc - \n{np.mean(values, ):.2f}', xbins=dict(size = 10, start = start, end = end)), row=row, col=col)

        fig.update_layout(
            #title=f'Run-time distribution {API_name} ({time_type})',
            xaxis_title='millis',
            yaxis_title='count',
            barmode='overlay',
            bargap=0.1
        )

        fig.update_traces(opacity=0.75)
    else: 
        fig.add_trace(go.Histogram(x=data_list[par_proc], xbins=dict(size = 10, start = start, end = end)), row = par_proc + 1, col=1)




In [171]:
fig7 = make_subplots(rows=7, subplot_titles=[f'{i} proc mean = {np.mean(bash_obs_list[i])}' for i in range(7)], shared_xaxes=True)
for i in range(7):
    main_plot(fig7, bash_obs_list, "Bash", "obs", i)
fig7.update_layout(
    xaxis_title='millis',
    yaxis_title='count',
    bargap=0.1,
    font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    )
    )
fig7.update_traces(showlegend=False)
fig7.update_annotations(font_size=12)
fig7.write_image("plot4.pdf", scale = 1.0, width = 560, height = 650)
fig7.show()


In [172]:
fig8 = make_subplots(rows=2, subplot_titles=[f'Run-time distribution Bash ({i})' for i in ['rep', 'obs']], shared_xaxes=True)
main_plot(fig8, bash_rep_list, "Bash", "rep", col=1, row=1)
main_plot(fig8, bash_obs_list, "Bash", "obs", col=1, row=2)
fig8.update_layout( 
    height = 500,
    coloraxis=dict(colorscale='Bluered_r'),
    font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    )
    )
fig8.update_annotations(font_size=12)
fig8.write_image("plot5.pdf", scale = 1.0, width = 600, height = 400)
fig8.show()


In [215]:
case_folder = 'Usefull_data'

relb_files = [f'{i}par_ntbrelb.pkl' for i in range(7)]
fish_files = [f'{i}par_ntbfish.pkl' for i in range(7)]
time_files = [f'{i}par_ntbtime.pkl' for i in range(7)]
bash_files = [f'{i}par_ntbbash.pkl' for i in range(7)]

fish_rep_list, fish_obs_list = unpack_data(fish_files)
time_rep_list, time_obs_list = unpack_data(time_files)
bash_rep_list, bash_obs_list = unpack_data(bash_files)
relb_rep_list, relb_obs_list = unpack_data(relb_files)
observed_times = [relb_obs_list, fish_obs_list, bash_obs_list, time_obs_list]
reported_times = [relb_rep_list, fish_rep_list, bash_rep_list, time_rep_list]

In [200]:
fig = make_subplots(rows=4, subplot_titles=labels, shared_xaxes=True)
fig.update_layout(font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    ))
quadro_plot(fig, observed_times, labels, "obs", end = 2400)
fig.update_annotations(font_size=12)
fig.write_image("plot6.pdf", scale = 1.0, width = 550, height = 680)
fig.show()

In [7]:
case_folder = 'Usefull_data'

relb_files = [f'{i}par_relb.pkl' for i in range(7)]
fish_files = [f'{i}par_fish.pkl' for i in range(7)]
time_files = [f'{i}par_time.pkl' for i in range(7)]
bash_files = [f'{i}par_bash.pkl' for i in range(7)]

fish_rep_list, fish_obs_list = unpack_data(fish_files)
time_rep_list, time_obs_list = unpack_data(time_files)
bash_rep_list, bash_obs_list = unpack_data(bash_files)
relb_rep_list, relb_obs_list = unpack_data(relb_files)
observed_times = [relb_obs_list, fish_obs_list, bash_obs_list, time_obs_list]
reported_times = [relb_rep_list, fish_rep_list, bash_rep_list, time_rep_list]

In [8]:
fig = make_subplots(rows=4, subplot_titles=labels, shared_xaxes=True)
fig.update_layout(font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    ))
quadro_plot(fig, observed_times, labels, "obs")
fig.update_annotations(font_size=12)
fig.write_image("plot3.pdf", scale = 1.0, width = 550, height = 680)
fig.show()

In [20]:
import plotly.io as pio   
pio.kaleido.scope.mathjax = None

In [166]:
fig7 = make_subplots(rows=7, subplot_titles=[f'{i} proc mean = {np.mean(bash_obs_list[i])}' for i in range(7)], shared_xaxes=True)
for i in range(7):
    main_plot(fig7, bash_obs_list, "Bash", "obs", i, start = 1000, end = 6000)
fig7.update_layout(
    xaxis_title='millis',
    yaxis_title='count',
    bargap=0.1,
    font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    )
    )
fig7.update_traces(showlegend=False)
fig7.update_annotations(font_size=12)
fig7.write_image("plot1.pdf", scale = 1.0, width = 550, height = 600)
fig7.show()


In [180]:
fig1 = make_subplots(rows=2, subplot_titles=[f'Run-time distribution Bash ({i})' for i in ['rep', 'obs']], shared_xaxes=True)
main_plot(fig1, bash_rep_list, "Bash", "rep", col=1, row=1, start = 1000, end = 6000)
main_plot(fig1, bash_obs_list, "Bash", "obs", col=1, row=2, start = 1000, end = 6000)
fig1.update_layout( 
    coloraxis=dict(colorscale='Bluered_r'),
    font=dict(
        size=12
    ),
    margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=20, #top margin
    )
)
fig1.update_annotations(font_size=12)
fig1.write_image("plot2.pdf", scale = 1.0, width = 550, height = 357)
fig1.show()